In [3]:
import csv
import pymssql
from itertools import islice
from time_util import str_to_timestamp

import os
import re

def parse_data(filepath):
    f = open(filepath)
    data = []
    for line in islice(csv.reader(f), 1, None):
        timeStr = line[0]
        ask = line[1]
        bid = line[2]
        ask_volume = line[3]
        bid_volume = line[4]
        timestamp = str_to_timestamp(timeStr)
        info = (int(timestamp/1000000), ask, bid, ask_volume, bid_volume)
        data.append(info)
    return data

def insert_by_filepath(filepath, table_name):
    data = parse_data(filepath)

    sql_drop_table = f"IF exists (select * from sys.objects where object_id = object_id('{table_name}') AND type ='U') " \
                     f"drop table {table_name};"
    cursor.execute(sql_drop_table)

    sql_create_table = f"""create table {table_name} (
                              id int identity(1,1) not null,
                              time bigint not null,
                              ask decimal(20,10) not null,
                              bid decimal(20,10) not null,
                              ask_volume decimal(20,10) not null,
                              bid_volume decimal(20,10) not null,
                              primary key (id)
                            )"""
    cursor.execute(sql_create_table)

    # mssql一次最多插入1000行数据，因此要对数据进行拆分
    step = 1000
    sub_datas = [data[i:i + step] for i in range(0, len(data), step)]
    rows = 0
    for sub_data in sub_datas:
        sql = f"insert into {table_name}(time, ask, bid, ask_volume, bid_volume) values "
        params = ()
        for info in sub_data:
            sql += "(%s,%s,%s,%s,%s),"
            params += info
        sql = sql.rstrip(",")
        cursor.execute(sql, params)
        rows += cursor.rowcount
        print(f"\r{table_name} 已插入{rows}/{len(data)}行", end="", flush=True)
    conn.commit()
    print()

    sql_create_index_time = f"create index {table_name}_time on {table_name}(time);"
    cursor.execute(sql_create_index_time)

    print(table_name)
    sql_create_index_ask = f"create index {table_name}_ask on {table_name}(ask);"
    cursor.execute(sql_create_index_ask)

    sql_create_index_bid = f"create index {table_name}_bid on {table_name}(bid);"
    cursor.execute(sql_create_index_bid)

def import_XAUUSD():
    dir = r"C:\Users\Dell\Desktop\XAUUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"XAUUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_xauusd_nf_{year}{month}{day}"
        insert_by_filepath(filepath, table_name)
        
def import_USCPI():
    dir = r"C:/Users/Dell/Desktop/XAUUSD_Ticks_USCPI"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"XAUUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_xauusd_uscpi_{year}{month}{day}"
        insert_by_filepath(filepath, table_name)
        
def import_FOMC():
    dir = r"C:\Users\Dell\Desktop\XAUUSD_Ticks_FOMC"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"XAUUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_xauusd_fomc_{year}{month}{day}"
        insert_by_filepath(filepath, table_name)

def import_CMDUSD():
    dir = r"C:\Users\Dell\Desktop\CMDUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"LIGHT.CMDUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_cmdusd_eia_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)
        
def import_USDJPY():
    dir = r"C:\Users\Dell\Desktop\USDJPY"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"USDJPY_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_usdjpy_boj_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)
        
def import_GBPUSD():
    dir = r"C:\Users\Dell\Desktop\GBPUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"GBPUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_gbpusd_boe_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)

def import_EURUSD():
    dir = r"C:\Users\Dell\Desktop\EURUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"EURUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_eurusd_ecb_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)

def import_USDCAD():
    dir = r"C:\Users\Dell\Desktop\USDCAD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"USDCAD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_usdcad_boc_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)

def import_USATECH():
    dir = r"C:\Users\Dell\Desktop\USATECH"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"USATECH.IDXUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_usatech_nasdaq_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)

def import_EURJPY():
    dir = r"C:\Users\Dell\Desktop\EURJPY"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"EURJPY_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_eurjpy_gepmi_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)

def import_AUDUSD():
    dir = r"C:\Users\Dell\Desktop\AUDUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"AUDUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_audusd_rba_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)
        
def import_NZDUSD():
    dir = r"C:\Users\Dell\Desktop\NZDUSD"
    for filename in os.listdir(dir):
        filepath = os.path.join(dir, filename)
        pattern = r"NZDUSD_Ticks_(\d{2}).(\d{2}).(\d{4})-\d{2}.\d{2}.\d{4}.csv"
        match_obj = re.match(pattern, filename)
        if not match_obj:
            continue
        year = match_obj.group(3)
        month = match_obj.group(2)
        day = match_obj.group(1)
        table_name = f"ticks_nzdusd_rbnz_{year}{month}{day}"

        insert_by_filepath(filepath, table_name)
        
if __name__ == "__main__":
    conn = pymssql.connect(host='WIN-CPE38BHN8G8', user='DaUser1', password='Bdsk360360', database='ORG_DKMKT_DA', port=1433,
                           charset='utf8')
    cursor = conn.cursor()
    
#     import_USDCAD()
#     import_USDJPY()
#     import_XAUUSD()
#     import_CMDUSD()
#     import_USATECH()
#     import_AUDUSD()
#     import_EURJPY()
#     import_GBPUSD()
#     import_EURUSD()
#     import_NZDUSD()
#     import_FOMC()
#     import_USCPI()
    
